# SE395 Programming Assignment1
#### 201811118 이  구

In [ ]:
import gzip
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
import seaborn as sn

## 1. Prepare training/test dataset
### 1.1 Load MNIST Dataset

In [ ]:
import gzip
import numpy as np

def training_images():
    with gzip.open('train-images-idx3-ubyte.gz', 'r') as f:
    #with gzip.open('/content/gdrive/My Drive/MNIST_DATASET/train-images-idx3-ubyte.gz', 'r') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        image_count = int.from_bytes(f.read(4), 'big')
        row_count = int.from_bytes(f.read(4), 'big')
        column_count = int.from_bytes(f.read(4), 'big')
        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8).reshape((image_count, row_count, column_count))
        return images

def training_labels():
    with gzip.open('train-labels-idx1-ubyte.gz', 'r') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        label_count = int.from_bytes(f.read(4), 'big')
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

def test_images():
    with gzip.open('t10k-images-idx3-ubyte.gz', 'r') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        image_count = int.from_bytes(f.read(4), 'big')
        row_count = int.from_bytes(f.read(4), 'big')
        column_count = int.from_bytes(f.read(4), 'big')
        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8).reshape((image_count, row_count, column_count))
        return images
        
def test_labels():
    with gzip.open('t10k-labels-idx1-ubyte.gz', 'r') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        label_count = int.from_bytes(f.read(4), 'big')
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

def make_flatten_arr(arr):
    result = []
    for i in range(len(arr)):
        result.append(arr[i].flatten())
    return np.array(result)

def img_vis(X, Y, idx):
    img = X[idx].reshape(28,28)
    plt.imshow(img)
    plt.title(str(Y[idx]))

In [ ]:
# for train dataset
X_train = training_images()
X_train = make_flatten_arr(X_train)
Y_train = np.array(training_labels())

# for test dataset
X_test = test_images()
X_test = make_flatten_arr(X_test)
Y_test = test_labels()

# check result
print("--------- train set ---------")
print("Shape of X_train: ", X_train.shape)
print("Shape of Y_train: ", Y_train.shape)
print("\n--------- test set ---------")
print("Shape of X_test: ", X_test.shape)
print("Shape of Y_test: ", Y_test.shape)

In [ ]:
img_vis(X_train,Y_train, 15000)

### 1.2 Normalization

In [ ]:
X_train = (np.asfarray(X_train) / 255.0)
X_test = (np.asfarray(X_test) / 255.0)

## 2. Design a 5-layer Neural Network
### 2.1 Design sub-layers and back-propagation

#### 2.1.1 Linear layer

In [ ]:
class Linear:
    def __init__(self, input_size, output_size, learning_rate = 0.01):
        self.info = "Class for implementation of Linear Layer"
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.W = np.random.randn(input_size, output_size)*0.01
        self.b = np.zeros((1, output_size))

    def forward(self, S):
        return np.dot(S, self.W) + self.b 
        
    def backward(self, S, Z):
        gradient = np.dot(Z, self.W.T)
        delta_W = np.dot(S.T, Z)
        delta_b  = Z.mean(axis=0)*S.shape[0]
        self.W = self.W - self.learning_rate * delta_W  
        self.b = self.b - self.learning_rate * delta_b
        return gradient

#### 2.1.2 ReLU & LeakyReLU (LReLU) layer

In [ ]:
class ReLU:
    def __init__(self):
        self.info = "Class for implementation of ReLU Layer"

    def forward(self, S):
        return np.maximum(0,S)

    def backward(self, S, Z):
        Z[S <= 0] = 0
        return Z
        
class LReLU:
    def __init__(self, hyperparameter = 0.01):
        self.info = "Class for implementation of Leaky ReLU Layer"
        self.hyperparameter = hyperparameter
        
    def forward(self, S):
        return np.maximum(self.hyperparameter*S, S)
    
    def backward(self, S, Z):
        Z[S <= self.hyperparameter*S] = 0
        return Z

#### 2.1.3 SoftMax layer

In [ ]:
def softmax(a) :
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

#### 2.1.4 Cross-entropy loss 

In [ ]:
def softmax_crossentropy(result,Y, derivate=False):
    if derivate == False:
        answers = result[np.arange(len(result)),Y]
        cross_entropy = - answers + np.log(np.sum(np.exp(result),axis=-1))
        return cross_entropy
    else:
        answers = np.zeros_like(result)
        answers[np.arange(len(result)),Y] = 1
        softmax = np.exp(result) / np.exp(result).sum(axis=-1,keepdims=True)
        return (- answers + softmax) / result.shape[0]


def grad_softmax_crossentropy(result,Y):
    ones_for_answers = np.zeros_like(result)
    ones_for_answers[np.arange(len(result)),Y] = 1    
    softmax = np.exp(result) / np.exp(result).sum(axis=-1,keepdims=True)
    return (- ones_for_answers + softmax) / result.shape[0]

#### 2.1.5 SGD
 Linear layer의 weights, bias update는 2.1.1 Linear class의 backward 함수에서 구현하였고, training data를 random한 batch로 나누는 과정은 Model class 의 train 함수 내부에 구현하였습니다.

### 2.2 Design two 3-layer Neural Network

#### 2.2.1 
Sequence: Input - Linear-ReLU - Linear-ReLU - Linear-SoftMax   
The input and output size of NN: input 28x28, output 10   
Layer Design: Linear(784,128) ㅡ> ReLU() ㅡ>  Linear(128,64) ㅡ> ReLU() ㅡ> Linear(64,10)

#### 2.2.2 
Sequence: Input - Linear- LReLU - Linear- LReLU - Linear-SoftMax   
The input and output size of NN: input 28x28, output 10    
Layer Design: Linear(784,128) ㅡ> LReLU() ㅡ> Linear(128,64) ㅡ> LReLU() ㅡ> Linear(64,10) 

## 3. Design training process and train the network

In [ ]:
class Model():
    def __init__(self, layers, num_epoch = 5000, batch_size = 512):
        self.model = layers
        self.epochs = num_epoch
        self.batch_size = batch_size
        self.train_loss_log = []
        self.test_loss_log = []

    def train(self, X_train, Y_train, X_test, Y_test):
        model = self.model
        for epoch in range(self.epochs):
            for_minibatch = list(zip(X_train, Y_train))
            random.shuffle(for_minibatch)
            X, Y = zip(*for_minibatch)
            X = list(X)
            Y = list(Y)
            batches = []
            for i in range(0, len(Y), self.batch_size):
                start = i
                end = i+self.batch_size
                batch = []
                if end >= len(Y_train):
                    end = len(Y_train)
                batch.append(X[start:end])
                batch.append(Y[start:end])
                batches.append(batch)

            for batch in batches:
                X_batch = np.array(batch[0])
                Y_batch = np.array(batch[1])
                result_sqc = self.forward(X_batch)
                layer_inputs = [X_batch]+result_sqc
                result = result_sqc[-1]
                loss = softmax_crossentropy(result,Y_batch)
                loss_grad = grad_softmax_crossentropy(result,Y_batch)
                # print("loss_grad", loss_grad.shape)
                # print("layer_inputs[layer_index]",layer_inputs[-1].shape)
                for layer_index in range(len(model))[::-1]:
                    layer = model[layer_index]
                    loss_grad = layer.backward(layer_inputs[layer_index],loss_grad)
            #print("[after "+str(epoch+1)+"/"+str(self.epochs)+" epoch]", "train loss: ", self.calc_loss(X_train, Y_train),"validation loss: ",  self.calc_loss(X_test, Y_test)  )
            self.train_loss_log.append(self.calc_loss(X_train, Y_train))
            self.test_loss_log.append(self.calc_loss(X_test, Y_test))
                    
    def forward(self, X):
        result_sqc = []
        input = X
        for layer in self.model:
            result_sqc.append(layer.forward(input))
            input = result_sqc[-1]
        return result_sqc

    def predict(self, X):
        # for prob, softmax layer!
        result = self.forward(X)[-1]
        prob = softmax(result)
        predict_num = prob.argmax(axis=-1)
        return predict_num, prob
    
    def calc_loss(self, X, Y):
        layer_activations = self.forward(X)
        terminal_weights = layer_activations[-1]
        loss = softmax_crossentropy(terminal_weights,Y)
        return np.mean(loss)

    def calc_acc(self, X, Y):
        forward_result =self.forward(X)[-1]
        predict_list = forward_result.argmax(axis=-1)
        acc = np.mean(predict_list==Y)
        return acc

    def loss_log_visualization(self):
        plt.plot(self.train_loss_log, label='train loss')
        plt.plot(self.test_loss_log, label='validation loss')
        plt.xlabel("num iteration")
        plt.ylabel("loss")
        plt.legend(fontsize='x-large')
    
    def confusion_matrix_visualization(self, X_test, Y_test):
        pred_list, prob_list = self.predict(X_test)
        result = confusion_matrix(pred_list, Y_test.T, normalize="pred")
        sn.heatmap(result, annot=True, fmt='.2f', cmap="Blues")
        plt.xlabel("Predicted label")
        plt.ylabel("True label")

In [ ]:
layers = [Linear(784,128), ReLU(), Linear(128,64),ReLU(), Linear(64,10)]
Model_ReLU = Model(layers)
Model_ReLU.train(X_train, Y_train.T, X_test, Y_test.T)

In [ ]:
layers = [Linear(784,128), LReLU(), Linear(128,64), LReLU(), Linear(64,10)]
Model_LReLU = Model(layers)
Model_LReLU.train(X_train, Y_train.T, X_test, Y_test.T)

## 4. Test the network on the test data and visualization the results on the report

### 4.1 Show 10x10 confusion matrix

In [ ]:
Model_ReLU.confusion_matrix_visualization(X_test, Y_test)

In [ ]:
Model_LReLU.confusion_matrix_visualization(X_test, Y_test)

### 4.2 Show top 3 scored images with probability (for each class)

In [ ]:
def predict_top3(model, X):
    result = model.forward(X)[-1]
    prob = softmax(result)
    predict_num = prob.argmax(axis=-1)
    return predict_num, prob

def top3_scored_images_visualization(model, X_test, Y_test, desired_num):
    class_desired_num = []
    result_class, result_prob = predict_top3(model, X_test)
    for i in range(len(Y_test)):
        if result_class[i]==Y_test[i]: 
            if result_class[i]==desired_num:
                idx = i
                prob = result_prob[i][desired_num] * 100
                temp = (prob, idx)
                class_desired_num.append(temp)
        else:
            continue
    class_desired_num.sort(key = lambda element : element[0])
    
    #print(class_desired_num)
    fig, axes = plt.subplots(1, 3)
    k = 1
    for j in range(3):        
        idx = class_desired_num[-1-j][1]
        img  = X_test[idx].reshape(28,28)
        prob = class_desired_num[-1-j][0]
        plt.subplot(1,3,k)
        plt.imshow(img, cmap=plt.cm.gray)
        plt.title(str(prob)+"%")
        k += 1
    plt.suptitle('Top 3 Scored Image about '+ str(desired_num),fontsize=20)

In [ ]:
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 0)
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 1)
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 2)
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 3)
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 4)
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 5)
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 6)
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 7)
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 8)
top3_scored_images_visualization(Model_ReLU, X_test, Y_test, 9)

In [ ]:
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 0)
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 1)
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 2)
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 3)
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 4)
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 5)
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 6)
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 7)
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 8)
top3_scored_images_visualization(Model_LReLU, X_test, Y_test, 9)

### 4.3 Show training Loss graph

In [ ]:
Model_ReLU.loss_log_visualization()

In [ ]:
Model_LReLU.loss_log_visualization()

In [ ]:
print("Model_ReLU accuracy: ", Model_ReLU.calc_acc(X_test,Y_test))
print("Model_LReLU accuracy: ", Model_LReLU.calc_acc(X_test,Y_test))